Takes in .AWD, sleep log if it exists, calendar, and spits out some pretty pictures

In [1]:
import os,sys,inspect
current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0, '/data/MoodGroup/actigraphy/gavi/actigraPy') 

import actigraPy.actigraPy as act
import importlib
import numpy as np
import pandas as pd
from datetime import datetime
from datetime import timedelta

/data/MoodGroup/actigraphy/gavi/actigraPy/actigraPy/actigraPy.py:23: MatplotlibDeprecationWarning: 
The matplotlib.backends.tkagg module was deprecated in Matplotlib 3.0 and will be removed in 3.2.
  import matplotlib.backends.tkagg as tkagg


If you want to run the test data, run this cell instead of the one below

In [2]:
# data and log directories
sub = 'TEST' #subject NUMBER as STRING
sub_long = 'TEST'

out_dir = '/data/MoodGroup/actigraphy/gavi/actigraPy/data/output' #output directory
data_dir = '/data/MoodGroup/actigraphy/gavi/actigraPy/data'

if not os.path.exists(out_dir):
    os.mkdir(out_dir)


## Subject and directory info

In [3]:
fn = {}
#.AWD
awd_fn = os.path.join(data_dir, '%s.AWD'%sub_long)
#sleeplog
fn['cal']= os.path.join(data_dir, '%s_calendar_log.xls'%sub_long)
#calendar
fn['log'] = os.path.join(data_dir, '%s_sleeplog.xls'%sub_long)
logs = []

if os.path.isfile(awd_fn):
    for name in fn.keys():
        if os.path.isfile(fn[name]):
            print("%s has %s file at %s"%(sub,name,fn[name]))
            logs.append(fn[name])
        else:
            print("no " + name)
            fn[name] = ''
else:
    print("DO NOT CONTINUE!! THERE IS NO AWD FILE")

TEST has cal file at /data/MoodGroup/actigraphy/gavi/actigraPy/data/TEST_calendar_log.xls
TEST has log file at /data/MoodGroup/actigraphy/gavi/actigraPy/data/TEST_sleeplog.xls


### Make Mtimes file

Figure out where to clip awd file

In [4]:
#read AWD file
awd_dat = act.read_AWD(awd_fn)
idx={'start':[0],'end':[len(awd_dat['dt_list'])-1]}
#get the start and stops from each log if they exist
for log in logs:
    print(log)
    log_dat, kw_dat, comments = act.read_log(log,awd_dat)
    
    if 'watch_on' in kw_dat.keys():
        on_date = kw_dat['watch_on'].iloc[0]['OnDate']
        on_time = kw_dat['watch_on'].iloc[0]['OnTime']
        on = datetime(on_date.year,on_date.month,on_date.day,on_time.hour,on_time.minute)
        #check if that time is in dt_list:
        if awd_dat['dt_list'].count(on) > 0:
            on_idx=awd_dat['dt_list'].index(on)
            idx['start'].append(on_idx)

    if 'watch_off' in kw_dat.keys():
        off_date = kw_dat['watch_off'].iloc[0]['OffDate']
        off_time = kw_dat['watch_off'].iloc[0]['OffTime']
        off = datetime(off_date.year,off_date.month,off_date.day,off_time.hour,off_time.minute)
        if awd_dat['dt_list'].count(off) > 0:
            off_idx=awd_dat['dt_list'].index(off)
            idx['end'].append(off_idx)
 
idx

/data/MoodGroup/actigraphy/gavi/actigraPy/data/TEST_calendar_log.xls
     OffDate   OffTime     OnDate    OnTime                Comment
0        NaT       NaN 2016-02-08  10:00:00                  start
1 2016-02-07  13:00:00 2016-02-07  14:30:00  Fake, for bug testing
2 2016-02-08  13:00:00 2016-02-08  14:30:00                     3T
3 2016-02-08  21:30:00 2016-02-09  07:00:00            Sleep study
4 2016-02-09  14:00:00 2016-02-09  16:00:00                     7T
5 2016-02-10  10:00:00 2016-02-10  12:00:00                    MEG
6 2016-02-10  21:30:00 2016-02-11  07:00:00            Sleep study
7 2016-02-11  09:00:00 2016-02-11  09:40:00               Infusion
8 2016-02-11  15:00:00 2016-02-11  17:00:00                    MEG


UnboundLocalError: local variable 'pos' referenced before assignment

In [ ]:
start = max(idx['start'])
end = min(idx['end'])
print('start = %d, end = %d'%(start,end))

## <font color='red'>Make sure the starts and ends make sense before clipping the data</font>


In [5]:
#modify these according to output from above!!
lim = [awd_dat['DateTime'][start],awd_dat['DateTime'][end]]
clip_dat = act.clip_dat(lim,awd_dat)

NameError: name 'start' is not defined

Now just run these cells in order to extract comments and write Mtimes

Testing/playing w code

In [6]:
fn = fn['cal']

fn_pref,fn_ext = os.path.splitext(fn)
if fn_ext == '.csv':
    log_dat = pd.read_csv(fn, keep_default_na=False)
elif fn_ext == '.xls':
    log_dat = pd.read_excel(fn)

In [7]:
keywords = {'watch_on':['Start','start'],'watch_off':['End','end']}
kw_dat = {}
for ii in keywords.keys():
    val = np.where(log_dat.Comment.isin(keywords[ii]))[0]
       #print(ii,val)
    if len(val) > 0:
        kw_dat[ii] = (log_dat.iloc[val])
        log_dat.drop(log_dat.index[val],inplace=True)


In [8]:
log_dat['On'] = pd.to_datetime(log_dat['OnDate'].astype(str) +
                                   ' ' + log_dat['OnTime'].astype(str) )
log_dat['Off'] = pd.to_datetime(log_dat['OffDate'].astype(str) + ' ' + log_dat['OffTime'].astype(str) )
log_dat = log_dat.to_dict(orient='list')    


In [9]:
dt_fmt = '%d-%b-%y %I:%M %p'

st_time = np.array([ datetime.strftime(ii,dt_fmt) for ii in log_dat['Off'] ])
en_time = np.array([ datetime.strftime(ii,dt_fmt) for ii in log_dat['On'] ])

In [66]:
mk_time = list(zip(st_time, en_time))
mk_time


[('07-Feb-16 01:00 PM', '07-Feb-16 02:30 PM'),
 ('08-Feb-16 01:00 PM', '08-Feb-16 02:30 PM'),
 ('08-Feb-16 09:30 PM', '09-Feb-16 07:00 AM'),
 ('09-Feb-16 02:00 PM', '09-Feb-16 04:00 PM'),
 ('10-Feb-16 10:00 AM', '10-Feb-16 12:00 PM'),
 ('10-Feb-16 09:30 PM', '11-Feb-16 07:00 AM'),
 ('11-Feb-16 09:00 AM', '11-Feb-16 09:40 AM'),
 ('11-Feb-16 03:00 PM', '11-Feb-16 05:00 PM')]

In [88]:
mk_list = [[x,y,z] for x in st_time.tolist() for y in en_time.tolist() for z in log_dat['Comment']]

In [96]:
mk_list = []
for i in range(0, len(st_time)):
    tmp = [[st_time[i],en_time[i]],log_dat['Comment'][i]]
    mk_list.append(tmp)

In [99]:
mk_list=[[[st_time[i],en_time[i]],log_dat['Comment'][i]] for i in range(0,len(st_time))]

In [112]:
mk_tuple = list(zip(st_time,en_time,log_dat['Comment']))

In [113]:
mk_tuple

[('07-Feb-16 01:00 PM', '07-Feb-16 02:30 PM', 'Fake, for bug testing'),
 ('08-Feb-16 01:00 PM', '08-Feb-16 02:30 PM', '3T'),
 ('08-Feb-16 09:30 PM', '09-Feb-16 07:00 AM', 'Sleep study'),
 ('09-Feb-16 02:00 PM', '09-Feb-16 04:00 PM', '7T'),
 ('10-Feb-16 10:00 AM', '10-Feb-16 12:00 PM', 'MEG'),
 ('10-Feb-16 09:30 PM', '11-Feb-16 07:00 AM', 'Sleep study'),
 ('11-Feb-16 09:00 AM', '11-Feb-16 09:40 AM', 'Infusion'),
 ('11-Feb-16 03:00 PM', '11-Feb-16 05:00 PM', 'MEG')]

In [125]:
dat_time = list(awd_dat['DateTime'])
tmp_list = []
exist_list = []
for block in mk_tuple:
    tmp = ()
    exist = ()
    for time in block[0:2]:
        try:
            tmp = tmp+(dat_time.index(time),)
            exist=exist+(True,)
        except:
            if time<dat_time[0]:
                tmp=tmp+(0,)
            elif time>dat_time[-1]:
                tmp=tmp+(len(dat_time)-1,)
            exist=exist+(False,)
    tmp = tmp+(block[2],)
    exist_list.append(exist)
    tmp_list.append(tmp)

In [131]:
for idx,tup in enumerate(tmp_list):
    if (exist_list[idx] == (False,False)):
        tmp_list.pop(idx)

In [132]:
[x]

[(0, 74, '3T'),
 (494, 1064, 'Sleep study'),
 (1484, 1604, '7T'),
 (2684, 2804, 'MEG'),
 (3374, 3944, 'Sleep study'),
 (4064, 4104, 'Infusion'),
 (4424, 4544, 'MEG')]

In [138]:
[y for z in [x[0:2] for x in tmp_list] for y in z]

[0, 74, 494, 1064, 1484, 1604, 2684, 2804, 3374, 3944, 4064, 4104, 4424, 4544]

Real notebook continues here

In [ ]:
importlib.reload(act)
#Make master mk_idx
mk_idx,comments=act.get_markers(awd_dat)

In [ ]:
#Read logs
mc_dict = {} #for Mtimes
for name in fn.keys():
    if os.path.isfile(fn[name]):
        c_dict={}
        log_dat,kw_dat,comments_tmp = act.read_log(fn[name],awd_dat)
        mk_idx[name]=log_dat['idx']
        
        #process comments
        zipped_pairs = zip(comments_tmp[0].tolist(),comments_tmp[1])
        c_dict['comments'] = [x for _, x in sorted(zipped_pairs)]
        c_dict['idxs'] = np.sort(comments_tmp[0]).tolist()
        mc_dict[name]=c_dict

In [ ]:
importlib.reload(act)
act.write_Mtimes_new(awd_dat,mk_idx,os.path.join(out_dir,sub_long),mc_dict)

## Make graphs

In [ ]:
plots = {}
del_com={}
for name in fn.keys():
    if name in mk_idx.keys():
        plots[name]=mk_idx[name] - start
        del_com[name]=((np.where(plots[name]<0)[0])//2).tolist()[::2]
        plots[name]=plots[name][np.where(plots[name]>0)]

In [ ]:
coms=[]
idxs=[]
for mm in mc_dict.keys():
    print(mm)
    idxs= idxs +(mc_dict[mm]['idxs'])
    coms= coms + (mc_dict[mm]['comments'])
    if mm in del_com.keys():
        for i in range(0,len(del_com[mm])):
            idxs.pop(0)
            coms.pop(0)
zipped_pairs = zip(idxs,coms)
gc_coms = [x for _, x in sorted(zipped_pairs)]
gc_idx = np.sort(idxs) - start

gc=[np.asarray(gc_idx),gc_coms]

In [ ]:
plots['cal']

In [ ]:
importlib.reload(act)

act.plot_awd(clip_dat,plots,max_act=500,show=False,comments=gc,fn_pref=os.path.join(out_dir,sub_long+'_M+logs'),plot_type='single',debug=True)


In [ ]:
importlib.reload(act)

act.plot_awd(clip_dat,plots,max_act=250,show=False,comments=gc,fn_pref=os.path.join(out_dir,sub_long+'_M+logs_zoom'),plot_type='single',debug=True)
